# **Extracción, Transformación y Carga (ETL) - Dataset: Internet**

Se hará el proceso de ETL para el dataset "internet" del proyecto de Data Analyst en telecomunicaciones. Se inicia priorizando cuáles de las 15 hojas del dataset son relevantes para el proyecto.

## **1. Hojas relevantes**

Luego de una revisión se considera que para el fin del proyecto, el cual es la realización de un análisis completo que permita reconocer el comportamiento del sector de las telecomunicaciones a nivel nacional, solamente se tendrán en cuentas algunas páginas de el dataset de "*Internet*".

*   **Penetración-hogares**: Para el análisis de la penetración de internet en los hogares, esencial para medir el KPI sobre el aumento de acceso a internet.
*   **Accesos_por_velocidad:** Relevante para identificar patrones de uso según la velocidad de conexión, lo que puede ayudar en el análisis del servicio brindado.
*  **Accesos_tecnologia_localidad:** Esta hoja puede ser útil para entender la distribución del acceso a internet por tecnologías y localidades, y encontrar oportunidades de mejora.
* **Ingresos:** Datos importantes para relacionar el crecimiento del servicio con los ingresos generados, que también puede servir para evaluar el desempeño financiero.

## **2. Carga de la base de datos**

In [ ]:
import pandas as pd

# Cargamos el dataset
datos = '/content/Internet.xlsx'

In [ ]:
# Seleccionamos las hojas
hojas = ['Penetracion-hogares', 'Accesos por velocidad', 'Accesos_tecnologia_localidad', 'Ingresos ']

In [ ]:
# Cargamos cada hoja en un diccionario de DataFrames
dataframes_hojas = {hoja: pd.read_excel(datos, sheet_name=hoja) for hoja in hojas}

In [ ]:
# Exploramos la estructura de los datos cargados
for hoja, df in dataframes_hojas.items():
    print("###################################################################")
    print(f"Hoja: {hoja}")
    print("###################################################################")
    print(df.info())
    print("###################################################################")
    print(df.head())

###################################################################
Hoja: Penetracion-hogares
###################################################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           984 non-null    int64  
 1   Trimestre                     984 non-null    int64  
 2   Provincia                     984 non-null    object 
 3   Accesos por cada 100 hogares  984 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 30.9+ KB
None
###################################################################
    Año  Trimestre        Provincia  Accesos por cada 100 hogares
0  2024          1     Buenos Aires                         81.10
1  2024          1  Capital Federal                        119.53
2  2024          1        Catamarca           

## **3. Datos faltantes**

Procedemos a identificar las columnas con valores faltantes y decidir cómo tratarlos (imputación, eliminación, etc.).

In [ ]:
# Verificamos valores faltantes en cada hoja
for hoja, df in dataframes_hojas.items():
    print(f"Hoja: {hoja}")
    print(df.isnull().sum())
    print("###################################################################")

Hoja: Penetracion-hogares
Año                             0
Trimestre                       0
Provincia                       0
Accesos por cada 100 hogares    0
dtype: int64
###################################################################
Hoja: Accesos por velocidad
Año                    0
Trimestre              0
Provincia              0
HASTA 512 kbps         0
+ 512 Kbps - 1 Mbps    0
+ 1 Mbps - 6 Mbps      0
+ 6 Mbps - 10 Mbps     0
+ 10 Mbps - 20 Mbps    0
+ 20 Mbps - 30 Mbps    0
+ 30 Mbps              0
OTROS                  6
Total                  0
dtype: int64
###################################################################
Hoja: Accesos_tecnologia_localidad
Provincia        280
Partido          280
Localidad        280
Link Indec       280
ADSL               0
CABLEMODEM         0
DIAL UP            0
FIBRA OPTICA       0
OTROS              0
SATELITAL          0
WIMAX              0
WIRELESS           0
Total general      0
dtype: int64
###########################

### 3.1. Hoja: `Accesos por velocidad`

Tenemos que la columna `OTROS` tiene $6$ valores faltantes. Dado que estos faltantes representan apenas un $0.6 \; \%$ de los datos (el cual no consideramos una cantidad significativa), procederemos a eliminarlos.

In [ ]:
# Eliminamos las filas con valores faltantes en la columna 'OTROS' de la hoja 'Accesos por velocidad'
dataframes_hojas['Accesos por velocidad'].dropna(subset=['OTROS'], inplace=True)

### 3.2. Hoja: `Accesos_tecnologia_localidad`

En esta hoja se procede a revisar mas a detalle ya que se tienen $280$ valores faltantes, se encuentra que esos nulos aparecen porque hay filas vacias, pero en ciertas columnas estan rellenas con el caracter guión "-", luego no contienen ningún tipo de información, así que vamos a eliminar las filas que contienen esos supuestos datos faltantes.

In [ ]:
# Eliminamos las filas con valores faltantes en 'Accesos_tecnologia_localidad'
dataframes_hojas['Accesos_tecnologia_localidad'].dropna(subset=['Provincia', 'Partido', 'Localidad', 'Link Indec'], inplace=True)

### 3.3. Confirmación de inexistencia de datos faltantes

In [ ]:
# Confirmamos que los valores faltantes han sido manejados
for hoja, df in dataframes_hojas.items():
    print(f"Hoja: {hoja}")
    print(df.isnull().sum())
    print("###################################################################")

Hoja: Penetracion-hogares
Año                             0
Trimestre                       0
Provincia                       0
Accesos por cada 100 hogares    0
dtype: int64
###################################################################
Hoja: Accesos por velocidad
Año                    0
Trimestre              0
Provincia              0
HASTA 512 kbps         0
+ 512 Kbps - 1 Mbps    0
+ 1 Mbps - 6 Mbps      0
+ 6 Mbps - 10 Mbps     0
+ 10 Mbps - 20 Mbps    0
+ 20 Mbps - 30 Mbps    0
+ 30 Mbps              0
OTROS                  0
Total                  0
dtype: int64
###################################################################
Hoja: Accesos_tecnologia_localidad
Provincia        0
Partido          0
Localidad        0
Link Indec       0
ADSL             0
CABLEMODEM       0
DIAL UP          0
FIBRA OPTICA     0
OTROS            0
SATELITAL        0
WIMAX            0
WIRELESS         0
Total general    0
dtype: int64
#####################################################

## **4. Datos duplicados**

Primero identificaremos si existen datos duplicados en cada hoja.
En caso de encontralos procederemos a eliminarlos.

In [ ]:
# Verificamos y eliminamos duplicados en cada hoja del diccionario
for hoja, df in dataframes_hojas.items():
    print(f"Antes de eliminar duplicados en {hoja}: {df.duplicated().sum()} filas duplicadas")

    # Eliminamos duplicados
    dataframes_hojas[hoja] = df.drop_duplicates()

    print(f"Después de eliminar duplicados en {hoja}: {dataframes_hojas[hoja].duplicated().sum()} filas duplicadas")
    print("###################################################################")


Antes de eliminar duplicados en Penetracion-hogares: 0 filas duplicadas
Después de eliminar duplicados en Penetracion-hogares: 0 filas duplicadas
###################################################################
Antes de eliminar duplicados en Accesos por velocidad: 0 filas duplicadas
Después de eliminar duplicados en Accesos por velocidad: 0 filas duplicadas
###################################################################
Antes de eliminar duplicados en Accesos_tecnologia_localidad: 0 filas duplicadas
Después de eliminar duplicados en Accesos_tecnologia_localidad: 0 filas duplicadas
###################################################################
Antes de eliminar duplicados en Ingresos : 0 filas duplicadas
Después de eliminar duplicados en Ingresos : 0 filas duplicadas
###################################################################


Como podemos apreciar, no habían datos duplicados.

## **5. Outliers**

Usaremos el método del rango intercuartílico (IQR) para identificar posibles outliers en las columnas numéricas.

El método del rango intercuartílico (IQR) es una técnica comúnmente utilizada para identificar valores atípicos (outliers) en columnas numéricas. El $IQR$ es la diferencia entre el tercer cuartil ($Q_3$) y el primer cuartil ($Q_1$) de los datos. Los cuartiles dividen los datos en partes iguales, donde:

*   $Q_1$: Es el valor que deja por debajo el $25\%$ de los datos.
*   $Q_3$: Es el valor que deja por debajo el $75\%$ de los datos.
*   $IQR$ (rango intercuartílico): Es la diferencia entre el tercer y el primer cuartil. Representa la amplitud de la "zona central" de los datos, es decir, el rango donde se encuentra el $50\%$ central de los datos.



In [ ]:
# Creamos una función para detectar outliers usando el método del IQR
def detecta_outliers(df, column):
    """
    Detecta outliers en una columna numérica de un DataFrame usando el método del rango intercuartílico (IQR).

    Parámetros:
    df (pandas.DataFrame): El DataFrame que contiene los datos.
    column (str): El nombre de la columna en la cual se quieren detectar los outliers.

    Retorna:
    pandas.DataFrame: Un DataFrame que contiene únicamente las filas donde los valores de la columna son outliers.

    El método del IQR identifica outliers como aquellos valores que están por debajo de Q1 - 1.5 * IQR o
    por encima de Q3 + 1.5 * IQR, donde Q1 es el primer cuartil (25%) y Q3 es el tercer cuartil (75%).
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

In [ ]:
# Detectamos outliers en las columnas numéricas de cada hoja
for hoja, df in dataframes_hojas.items():
    print(f"Hoja: {hoja}")
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    for col in numeric_cols:
        outliers = detecta_outliers(df, col)
        print(f"Columna: {col}, Outliers encontrados: {len(outliers)}")
        if len(outliers) > 0:
            print(outliers.head())  # Mostrar algunos outliers para inspección
    print("###################################################################")

Hoja: Penetracion-hogares
Columna: año, Outliers encontrados: 0
Columna: trimestre, Outliers encontrados: 0
Columna: accesos_por_cada_100_hogares, Outliers encontrados: 20
     año  trimestre        provincia  accesos_por_cada_100_hogares
1   2024          1  Capital Federal                        119.53
25  2023          4  Capital Federal                        117.02
49  2023          3  Capital Federal                        116.40
73  2023          2  Capital Federal                        115.48
97  2023          1  Capital Federal                        122.59
###################################################################
Hoja: Accesos por velocidad
Columna: año, Outliers encontrados: 0
Columna: trimestre, Outliers encontrados: 0
Columna: hasta_512_kbps, Outliers encontrados: 167
     año  trimestre     provincia  hasta_512_kbps  _512_kbps__1_mbps  \
0   2024          1  Buenos Aires         26002.0            22510.0   
24  2024          4  Buenos Aires         26909.0    

### 5.1. Hoja `Penetracion-hogares`

En la columna `Accesos por cada 100 hogares`, los outliers están asociados a la provincia Capital Federal, que tiene un acceso a internet mucho más alto que otras provincias.


Estos outliers reflejan una diferencia real, ya que es razonable esperar que en Capital Federal, al ser un área urbana con mejor infraestructura, los accesos a internet sean significativamente más altos.

### 5.2. Hoja `Accesos por velocidad`

Varias columnas presentan outliers:

* `HASTA 512 kbps`, `+ 512 Kbps - 1 Mbps`, `+ 1 Mbps - 6 Mbps`, etc.: Estos outliers parecen estar concentrados en provincias grandes como Buenos Aires, con una gran cantidad de accesos. Estos outliers podrían ser útiles para detectar diferencias regionales en el acceso a internet.

* También se pueden apreciar outliers en provincias más pequeñas, como Santa Fe, que reflejan un patrón interesante que podría analizarse más adelante.

### 5.3. Hoja `Accesos_tecnologia_localidad`

Outliers en casi todas las columnas (`ADSL`, `Cablemódem`, `DIAL UP`, entre otras):
Los outliers indican localidades con muy altos o muy bajos accesos para cada tecnología. Estas diferencias podrían ser clave para entender patrones tecnológicos en diferentes regiones.

### 5.4. Hoja `Ingresos`

* Columna `Año`: Hay un outlier donde el año es 2033 seguramente es un error tipográfico. Podríamos corregir este valor a 2023 para mantener la consistencia.

* Columna `Ingresos` (miles de pesos): Los outliers corresponden a ingresos significativamente más altos en trimestres recientes. Estos outiers probablemente reflejan un crecimiento real en la industria de las telecomunicaciones.

In [ ]:
# Corregimos el año 2033 en la hoja 'Ingresos'
dataframes_hojas['Ingresos '].loc[dataframes_hojas['Ingresos ']['Año'] == 2033, 'Año'] = 2023


## **6. Transformación de los datos**

### 6.1. Revisión de formatos de fechas

Para poder verificar si tenemos columnas con información de fechas en todas las hojas del dataset, podemos buscar las columnas que contengan datos relacionados con fechas (en este caso, aquellas que podrían contener el año, mes o periodo) y revisar si están en el formato correcto. Para ello creamos la siguiente función

In [ ]:
# Función para identificar columnas que podrían contener fechas
def revisar_columnas_fechas(df):
    """
    Identifica columnas en un DataFrame que potencialmente contienen fechas basándose en los nombres de las columnas.

    Parámetros:
    df (pandas.DataFrame): El DataFrame que contiene los datos.

    Retorna:
    dict: Un diccionario donde las claves son los nombres de las columnas potencialmente relacionadas con fechas,
    y los valores indican si el formato es correcto (es decir, ya es un formato de fecha) o si necesita conversión.

    El criterio para identificar columnas de fechas es revisar si en el nombre de la columna aparece 'fecha', 'periodo'
    o 'año'. Luego, se intenta convertir los valores de esas columnas al formato datetime para verificar su validez.
    """
    # Identificar columnas que potencialmente contengan fechas: aquellas con 'fecha', 'periodo' o 'año' en su nombre
    columnas_potenciales_fechas = [col for col in df.columns if 'fecha' in col.lower() or 'periodo' in col.lower() or 'año' in col.lower()]

    # Verificar si las columnas ya están en formato de fecha o si necesitan conversión
    verificacion_fechas = {}
    for columna in columnas_potenciales_fechas:
        # Intentar convertir a datetime y verificar si hay errores
        try:
            pd.to_datetime(df[columna], errors='raise')
            verificacion_fechas[columna] = 'Formato de fecha correcto'
        except:
            verificacion_fechas[columna] = 'Formato incorrecto o texto'

    return verificacion_fechas


In [ ]:
# Verificamos todas las hojas para identificar posibles columnas de fechas
for hoja, df in dataframes_hojas.items():
    print(f"Verificando fechas en la hoja: {hoja}")
    result = revisar_columnas_fechas(df)  # Usamos la función que creamos
    if result:
        print(result)
    else:
        print("No se encontraron columnas relacionadas con fechas.")
    print("###################################################################")


Verificando fechas en la hoja: Penetracion-hogares
{'Año': 'Formato de fecha correcto'}
###################################################################
Verificando fechas en la hoja: Accesos por velocidad
{'Año': 'Formato de fecha correcto'}
###################################################################
Verificando fechas en la hoja: Accesos_tecnologia_localidad
No se encontraron columnas relacionadas con fechas.
###################################################################
Verificando fechas en la hoja: Ingresos 
{'Año': 'Formato de fecha correcto', 'Periodo': 'Formato incorrecto o texto'}
###################################################################


<ipython-input-114-c387c5440ad9>:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df[columna], errors='raise')


En la hoja de `Ingresos`, la columna `Periodo` contiene información de fechas como texto. Vamos a convertirla a un formato de fecha adecuado:

In [ ]:
# Revisamos el formato de fecha que tiene periodo
dataframes_hojas['Ingresos ']['Periodo'].head()

,Periodo
0,Ene-Mar 2024
1,Oct-Dic 2023
2,Jul-Sept 2023
3,Jun-Mar 2023
4,Ene-Mar 2023


Los datos de texto en la columna `Periodo` están en un formato mixto con abreviaciones de meses y trimestres (por ejemplo, "Ene-Mar 2024" o "Jul-Sept 2023"). Necesitamos transformar manualmente estas cadenas de texto en un formato que `pd.to_datetime()` pueda reconocer.

In [ ]:
# Creamos una función para mapear los meses de texto a un rango adecuado de fechas (trimestres)
def convertir_periodo_a_fecha(periodo_str):
    # Diccionario de traducción de abreviaciones de meses
    trimestre_a_fecha = {
        'Ene-Mar': '-03-31',
        'Abr-Jun': '-06-30',
        'Jul-Sept': '-09-30',
        'Oct-Dic': '-12-31',
    }

    # Dividir el texto en mes y año
    for trimestre, fecha in trimestre_a_fecha.items():
        if trimestre in periodo_str:
            # Extraer el año
            year = periodo_str.split()[-1]
            # Construir la fecha
            return pd.to_datetime(year + fecha, format='%Y-%m-%d')

    return pd.NaT  # Si no se reconoce el formato, devolver NaT (nulo)


In [ ]:
# Aplicamos la función a la columna 'Periodo' en la hoja 'Ingresos'
dataframes_hojas['Ingresos ']['Periodo'] = dataframes_hojas['Ingresos ']['Periodo'].apply(convertir_periodo_a_fecha)

In [ ]:
# Verificamos si la conversión fue exitosa
dataframes_hojas['Ingresos ']['Periodo'].head()


,Periodo
0,2024-03-31
1,2023-12-31
2,2023-09-30
3,NaT
4,2023-03-31


El tipo de dato `datetime64[ns]` nos indica que la columna `Periodo` en la hoja `Ingresos` ya está en formato de fecha correctamente. Sin embargo tenemos un valor nulo, revisando notamos que hay iun error en ese trimerte, no es `Jun-Mar 2023`, si ni que debía ser `Abr-Jun 2023`.

In [ ]:
# Reemplazar el valor incorrecto "Jun-Mar 2023" por "Abr-Jun 2023" en la columna 'Periodo'
dataframes_hojas['Ingresos ']['Periodo'] = dataframes_hojas['Ingresos ']['Periodo'].replace('Jun-Mar 2023', 'Abr-Jun 2023')

# Reemplazamos el valor NaT con la fecha correcta "2023-06-30"
dataframes_hojas['Ingresos ']['Periodo'].fillna(pd.Timestamp('2023-06-30'), inplace=True)

# Verificamos los resultados finales
dataframes_hojas['Ingresos ']['Periodo'].head()



,Periodo
0,2024-03-31
1,2023-12-31
2,2023-09-30
3,2023-06-30
4,2023-03-31


### 6.2. Normalización de los nombres de las columas

Vamos a normalizar los nombres de columnas para evitar problemas en el análisis posterior evitando espacios o caracteres especiales.

In [ ]:
# Creamos una función para normalizar nombres de columnas
def normaliza_nombres_columnas(df):
    """
    Normaliza los nombres de las columnas en un DataFrame.

    Esta función realiza los siguientes pasos en los nombres de las columnas:
    - Elimina espacios en blanco al inicio y final.
    - Convierte todas las letras a minúsculas.
    - Reemplaza los espacios entre palabras con guiones bajos.
    - Elimina caracteres no alfanuméricos.

    Parámetros:
    df (pandas.DataFrame): El DataFrame cuyas columnas se van a normalizar.

    Retorna:
    pandas.DataFrame: El DataFrame con los nombres de columnas normalizados.
    """
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w]', '', regex=True)
    return df

In [ ]:
# Aplicamos la normalización a todas las hojas
for hoja, df in dataframes_hojas.items():
    normaliza_nombres_columnas(df)

In [ ]:
# Verificamos algunos resultados
for hoja, df in dataframes_hojas.items():
    print(f"Nombres de columnas en la hoja {hoja}:")
    print(df.columns)
    print("###################################################################")

Nombres de columnas en la hoja Penetracion-hogares:
Index(['año', 'trimestre', 'provincia', 'accesos_por_cada_100_hogares'], dtype='object')
###################################################################
Nombres de columnas en la hoja Accesos por velocidad:
Index(['año', 'trimestre', 'provincia', 'hasta_512_kbps', '_512_kbps__1_mbps',
       '_1_mbps__6_mbps', '_6_mbps__10_mbps', '_10_mbps__20_mbps',
       '_20_mbps__30_mbps', '_30_mbps', 'otros', 'total'],
      dtype='object')
###################################################################
Nombres de columnas en la hoja Accesos_tecnologia_localidad:
Index(['provincia', 'partido', 'localidad', 'link_indec', 'adsl', 'cablemodem',
       'dial_up', 'fibra_optica', 'otros', 'satelital', 'wimax', 'wireless',
       'total_general'],
      dtype='object')
###################################################################
Nombres de columnas en la hoja Ingresos :
Index(['año', 'trimestre', 'ingresos_miles_de_pesos', 'periodo'], d

 Como podemos apreciar los nombres de las columnas han sido normalizados correctamente, eliminando espacios, convirtiendo a minúsculas y reemplazando caracteres no alfanuméricos. Algunos ejemplos de la normalización son:

* `Penetracion-hogares`: `accesos por cada 100 hogares` → `accesos_por_cada_100_hogares`
* `Accesos por velocidad`: `+ 512 Kbps - 1 Mbps` → `_512_kbps__1_mbps`

### 6.3. Formato correcto en las columnas numéricas

Vamos a revisar si las columnas que deberían contener valores numéricos están en el formato adecuado (`int` o `float`). Si encontramos alguna columna numérica que esté en formato de texto o que tenga valores no numéricos, podemos intentar convertirla al formato correcto.

In [ ]:
# Hacemos un listado de las columnas que deben permanecer como texto en cada hoja
columnas_texto = {
    'Penetracion-hogares': ['provincia'],
    'Accesos por velocidad': ['provincia'],
    'Accesos_tecnologia_localidad': ['provincia', 'partido', 'localidad', 'link_indec'],
    'Ingresos ': []  # En este caso, todas las columnas deberían ser numéricas excepto 'Periodo', que ya está como fecha
}

In [ ]:
# Revisamos y convertimos las columnas numéricas a int o float si es necesario
for hoja, df in dataframes_hojas.items():
    print(f"Revisando tipos de datos en la hoja: {hoja}")

    # Identificamos las columnas que deberían ser numéricas y no están en la lista de texto
    columnas_a_convertir = df.select_dtypes(include=['object']).columns.difference(columnas_texto.get(hoja, []))

    for col in columnas_a_convertir:
        try:
            # Intentamos convertir la columna a numérico (int o float)
            df[col] = pd.to_numeric(df[col], errors='coerce')
            print(f"Columna {col} convertida exitosamente a formato numérico.")
        except Exception as e:
            print(f"Error al convertir la columna {col}: {e}")

    # Verificamos el tipo de dato final de cada columna
    print("Tipos de datos finales:")
    print(df.dtypes)
    print("###################################################################")



Revisando tipos de datos en la hoja: Penetracion-hogares
Tipos de datos finales:
año                               int64
trimestre                         int64
provincia                        object
accesos_por_cada_100_hogares    float64
dtype: object
###################################################################
Revisando tipos de datos en la hoja: Accesos por velocidad
Tipos de datos finales:
año                    int64
trimestre              int64
provincia             object
hasta_512_kbps       float64
_512_kbps__1_mbps    float64
_1_mbps__6_mbps      float64
_6_mbps__10_mbps     float64
_10_mbps__20_mbps    float64
_20_mbps__30_mbps    float64
_30_mbps             float64
otros                float64
total                float64
dtype: object
###################################################################
Revisando tipos de datos en la hoja: Accesos_tecnologia_localidad
Tipos de datos finales:
provincia        object
partido          object
localidad        object
li

Ahora las columnas están correctamente formateadas:

* Las columnas `provincia`, `partido`, `localidad` y `link_indec` permanecen con el formato texto (`object`), que es el tipo de dato correcto para esas columnas.

* Las columnas numéricas en todas las hojas están en formato `int` o `float`, según corresponda.

* La columna `periodo` en la hoja `Ingresos` está en formato de fecha (`datetime64[ns]`), que también es correcto.

Con estas pasos, hemos finalizado la etapa de transformación de datos, asegurándonos de que todos los valores estén en los formatos adecuados. Ahora los datos están completamente preparados para el análisis.

In [ ]:
# Crea un archivo ExcelWriter para guardar los DataFrames en diferentes hojas
#with pd.ExcelWriter('dataframes_hojas.xlsx') as writer:
  # Itera sobre el diccionario de DataFrames
  #for hoja, df in dataframes_hojas.items():
    # Guarda cada DataFrame en una hoja diferente del archivo Excel
    #df.to_excel(writer, sheet_name=hoja, index=False)

# Descarga el archivo .xlsx
#from google.colab import files
#files.download('dataframes_hojas.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>